<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

В данной работе нам нужно защитить данные клиентов страховой компании «Хоть потоп». Будет разработан такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность работы метода.

Задача - защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось, при этом подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
#В данной ячейке указаны все необходимые для работы библиотеки.
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
# напишем функцию для изучения датафреймов
def research(data):

    display(data.head())
    print(data.info()) 
    display(data.describe())
    print('Количество дубликатов', data.duplicated().sum())  

In [4]:
research(data)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Количество дубликатов 153


Проведём первичный анализ рассматриваемых данных.
В нашем распоряжении:
5000 строк данных. В данных указан индекс, пол, возраст, зарплата, количество членов семьи, наличие/отсутствие страховых выплат. Для названия столбцов использована кириллица, столбцы не соответствуют змеиному регистру. Колонки *возраст* и *зарплата* записаны в типе данных float (число с точкой).  

*Признаки:* **Пол, Возраст, Зарплата, Члены семьи**  

*Целевой признак:* **Страховые выплаты**

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

**Ответ:** R2 score не изменится.

**Обоснование:** Рассмотрим свойства матриц.  



* Правила раскрытия скобок.

$$
(AB)^{-1} = B^{-1}A^{-1}
$$

$$
(AB)^{T} = B^{T}A^{T}
$$

* Порядок умножения нескольких таблиц

$$
A(BC) = (AB)C
$$

* Произведение матрицы на единичную матрицу **E** равно самой матрице

$$
EA = AE = A
$$

* Формула для модели на преобразованных признаках, где P - случайная обратимая матрица

$$
a' = X'w' = (XP)w'
$$

* Умножение квадратной матрицы на обратную

$$ 
AA^{-1} = E
$$

* Признаки умножаются на обратимую матрицу **P**. 

$$
w_1 = ((XP)^TXP)^{-1}(XP)^T y
$$

* Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке

$$
w_1 = (P^TX^TXP)^{-1}P^TX^{T} y
$$

* Если умножать матрицу на обратную матрицу будет равенство с единичной матрицей

$$
w_1 = P^{-1}(X^TX)^{-1}X^{T}y
$$

* При умножении любой матрицы на еденичную матрицу произведение равно этой самой матрице
$$
w_1 = P^{-1}(X^TX)^{-1}X^{T}y
$$

$$
a_1 = XPw_1
$$
$$
a_1 = XPP^{-1}(X^TX)^{-1}X^{T}y
$$
$$
a_1 = X(X^TX)^{-1}X^{T}y
$$

Таким образом доказываем, что  

$$
a_1 = Xw = a
$$

## Алгоритм преобразования

**Алгоритм**

* Генерация обратимой матрицы, размеры которой соответсвуют количеству столбцов (4х4)
* Перемножение матрицы изначальных признаков на сгенерированную.
* Посчитать качество модели на матрице исходных данных
* Посчитать качество модели на умноженной матрице

**Обоснование**

Метрики R2 как на изменнённой так и на изначальной матриц должны быть равны. Это будет являться докозательством верности защиты персональных данных клиентов.

## Проверка алгоритма

In [5]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [6]:
matrix_gen = np.random.normal(size=(4, 4))
display(matrix_gen)

array([[-0.1404194 ,  0.36661958,  1.30966152,  2.20512606],
       [ 0.02961557,  0.01626047, -0.3823094 ,  0.93703432],
       [ 1.75201713,  1.45897715, -0.23053028, -0.63189237],
       [-0.98895085, -2.28709361,  0.03664371,  1.27072812]])

Построим обратную матрицу из сгенерированной матрицы.

In [7]:
matrix_reverse = np.linalg.inv(matrix_gen)
display(matrix_reverse)

array([[ 0.05382753, -0.30944362,  0.91252913,  0.58854682],
       [ 0.08307976,  0.44612232, -0.37251459, -0.65838033],
       [ 0.44539329, -1.16088712,  0.14214648,  0.15382046],
       [ 0.17857723,  0.59559412,  0.0356189 ,  0.05558221]])

Построим матрицу из изначальных признаков и перемножим на обратную матрицу

In [8]:
matrix = features.values
matrix_coded = matrix@matrix_reverse
features_coded = pd.DataFrame(matrix_coded, columns = features.columns)
display(features_coded)

,Пол,Возраст,Зарплата,Члены семьи
0,22095.146018,-57561.423812,7036.140538,7603.145488
1,16928.945388,-44092.593206,5384.466249,5814.947671
2,9355.668470,-24365.691899,2974.273191,3211.136688
3,18575.002156,-48398.433001,5919.756715,6400.598474
4,11627.145014,-30286.971759,3700.505291,3996.867975
...,...,...,...,...
4995,15903.223955,-41429.987446,5064.270223,5473.067035
4996,23341.611853,-60814.721151,7435.845760,8037.862899
4997,15100.851390,-39343.959615,4811.386673,5201.457245
4998,14566.778002,-37949.716680,4641.014014,5016.200058


Следующий этап - проверка линейной регрессии.

In [9]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
display('R2 Незашифрованной модели равен',  r2_score(target, predictions))

'R2 Незашифрованной модели равен'

0.4249455028666801

In [10]:
model = LinearRegression()
model.fit(features_coded, target)
predictions = model.predict(features_coded)
display('R2 Зашифрованной модели равен',  r2_score(target, predictions))

'R2 Зашифрованной модели равен'

0.4249455028666792

**Вывод**  

R2 Показатель после кодировки данных, в целом, не изменился. 